# PWM(From Axi Timer)

## Pulse Width Modulation Mode
#### Pulse Width Modulation (PWM) mode has the following programming sequence:
#### • The mode for both Timer 0 and Timer 1 must be set to Generate mode (bit MDT in the TCSR set to 0).

#### • The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
#### • The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0 signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these signals must be enabled in both timer/counters.
#### • The assertion level of the GenerateOut signals for both timers in the pair must be set to Active High.
#### • The counter can be set to count up or down.
- https://docs.amd.com/v/u/en-US/pg079-axi-timer
- https://pynq.readthedocs.io/en/v2.5/pynq_package/pynq.overlay.html
## Pinout
### PWM: PMOD Pin3

In [8]:
from pynq import Overlay
overlay = Overlay('MainDesign.bit')

In [ ]:
overlay?

In [12]:
print(overlay.ip_dict['axi_timer_0']['registers']) # Thank lord for the JSON format, how else can I gouge my eyes out

{'TCSR0': {'address_offset': 0, 'size': 32, 'access': 'read-write', 'description': 'Timer 0 Control and Status Register', 'fields': {'MDT0': {'bit_offset': 0, 'bit_width': 1, 'access': 'read-write', 'description': 'Timer 0 Mode\n0 - Timer mode is generate\n1 - Timer mode is capture\n'}, 'UDT0': {'bit_offset': 1, 'bit_width': 1, 'access': 'read-write', 'description': 'Up/Down Count Timer 0\n  0 - Timer functions as up counter\n  1 - Timer functions as down counter\n'}, 'GENT0': {'bit_offset': 2, 'bit_width': 1, 'access': 'read-write', 'description': 'Enable External Generate Signal Timer 0\n  0 - Disables external generate signal\n  1 - Enables external generate signal\n'}, 'CAPT0': {'bit_offset': 3, 'bit_width': 1, 'access': 'read-write', 'description': 'Enable External Capture Trigger Timer 0\n  0 - Disables external capture trigger\n  1 - Enables external capture trigger\n'}, 'ARHT0': {'bit_offset': 4, 'bit_width': 1, 'access': 'read-write', 'description': 'Auto Reload/Hold Timer 0.\

In [36]:
# utility functions for interactin with specific bits, not needed anymore after testing
def set_bit(value, bit):
    return value | (1 << bit)


# Set the specified bit to 0
def clear_bit(value, bit):
    return value & ~(1 << bit)


In [33]:
# Referencing axi_timer IP
PWM = overlay.axi_timer_0

# Extracting register addresses for later
TCSR0 = overlay.ip_dict['axi_timer_0']['registers']['TCSR0']
TCSR1 = overlay.ip_dict['axi_timer_0']['registers']['TCSR1']
TLR0 = overlay.ip_dict['axi_timer_0']['registers']['TLR0']
TLR1 = overlay.ip_dict['axi_timer_0']['registers']['TLR1']


# init configuration values for TSCR0 and TSCR1 registers respectively
temp_val_0, temp_val_1 = 0b1010010110, 0b1010010110


# # The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
# temp_val_0 = set_bit(temp_val_0, TCSR0_register['PWMA0']['bit_offset'])
# temp_val_1 = set_bit(temp_val_1, TCSR1_register['PWMA1']['bit_offset'])

# # The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# # signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# # signals must be enabled in both timer/counters
# temp_val_0 = set_bit(temp_val_0, TCSR0_register['GENT0']['bit_offset'])
# temp_val_1 = set_bit(temp_val_1, TCSR1_register['GENT1']['bit_offset'])

# # The counter can be set to count up or down. UDT
# temp_val_0 = set_bit(temp_val_0, TCSR0_register['UDT0']['bit_offset'])
# temp_val_1 = set_bit(temp_val_1, TCSR1_register['UDT1']['bit_offset'])

# # Setting Autoreload (ARHT0 = 1)
# temp_val_0 = set_bit(temp_val_0, TCSR0_register['ARHT0']['bit_offset'])
# temp_val_1 = set_bit(temp_val_1, TCSR1_register['ARHT1']['bit_offset'])

# # Enable timer (ENT0 = 1)
# temp_val_0 = set_bit(temp_val_0, TCSR0_register['ENT0']['bit_offset'])
# temp_val_1 = set_bit(temp_val_1, TCSR1_register['ENT1']['bit_offset'])

print(bin(temp_val_0))
print(bin(temp_val_1))

0b1010010110
0b1010010110


In [34]:
# COnfigurable PWM values 
def SetPWM(DutyCycle):
    if DutyCycle >= 100:
        print("Range is 0 - 99(in percentage)")
    else:
        _period_ = 20000  # 50Hz, 20ms

        period = int((_period_ & 0x0ffff) *100);
        pulse = int((DutyCycle & 0x07f)*period/100);
        print(f"period: {period}")
        print(f"pulse: {DutyCycle}%")

        PWM.write(TCSR0['address_offset'], temp_val_0)
        PWM.write(TCSR1['address_offset'], temp_val_1)
        PWM.write(TLR0['address_offset'], period)
        PWM.write(TLR1['address_offset'], pulse)

In [35]:
SetPWM(55)

period: 2000000
pulse: 55%
